# Imports and loading data from sample_data folder in colabs

In [ ]:
# !pip install git+https://github.com/Cris-CL/deep_seo.git

In [22]:
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Sequential

In [7]:
def clean_tags(txt):
    return txt.replace('<br>','').replace('<b>','').replace('</br>','').replace('</b>','').replace('"','').replace("'",'').replace(',','').replace('[','').replace(']','').replace('&amp','')

def featureclean(cols):
    return cols.replace("[]","na").strip("[]'").lower()


In [10]:
df = pd.read_csv('/content/sample_data/meta_cellphone_100k.csv')

df_new = df[['category',
            'description',
            'title',
            'brand',
            'feature',
            'imageURLHighRes',
            'main_ranking_3',
            'numb_rankings']]
df_new['clean_description'] = df_new['description'].map(clean_tags)
df_new['clean_feature'] = df_new['feature'].map(featureclean)
df_new['rank_binss'] = pd.cut(df_new['main_ranking_3'], bins = 10, labels=[i for i in range(1,11)],include_lowest=True).astype('str')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [5]:
df_new.head()

,category,description,title,brand,feature,imageURLHighRes,main_ranking_3,numb_rankings,clean_description,clean_feature,rank_binss
0,"['Cell Phones & Accessories', 'Accessories', '...","['', ""<b>[ 9H HARDNESS FOR MAXIMUM PROTECTION ...",[2-Pack] Google Pixel 2 XL Screen Protector Li...,Live2Pedal,['Specifically designed for Google Pixel 2 XL'...,['https://images-na.ssl-images-amazon.com/imag...,5235,3,9H HARDNESS FOR MAXIMUM PROTECTION 9H harne...,"specifically designed for google pixel 2 xl', ...",1
1,"['Cell Phones & Accessories', 'Accessories', '...",['Loopilops: ---Surface Hardness : 9H hardness...,Loopilops iPhone 8Plus/7Plus/6Plus/6s Plus Scr...,Loopilops,[],['https://images-na.ssl-images-amazon.com/imag...,23198,3,Loopilops: ---Surface Hardness : 9H hardness c...,na,3
2,"['Cell Phones & Accessories', 'Cases, Holsters...","[""<b>PLEASE NOTE:</b> <br> NOT FOR S5 ACTIVE. ...",PowerBear Samsung Galaxy S5 Extended Battery [...,PowerBear,[],['https://images-na.ssl-images-amazon.com/imag...,14390,3,PLEASE NOTE: NOT FOR S5 ACTIVE. The S5 extend...,na,2
3,"['Cell Phones & Accessories', 'Cases, Holsters...","[""<b>PLEASE NOTE:</b> <br> The S4 extended bat...",PowerBear Samsung Galaxy S4 Extended Battery [...,PowerBear,['GO BEYOND YOUR POWER: Increase your battery ...,['https://images-na.ssl-images-amazon.com/imag...,68243,2,PLEASE NOTE: The S4 extended battery is compa...,go beyond your power: increase your battery po...,7
4,"['Cell Phones & Accessories', 'Accessories', '...","['', ""Specifically designed for iWatch Smart W...",2 Pack Apple 38mm Watch Screen Protector (38mm...,Live2Pedal,['Precise laser cutting Live2Pedal screen prot...,['https://images-na.ssl-images-amazon.com/imag...,122,3,Specifically designed for iWatch Smart Watch ...,precise laser cutting live2pedal screen protec...,1


In [22]:
df_new.imageURLHighRes[0].replace('[','').replace(']','').replace("'",'').split(',')

['https://images-na.ssl-images-amazon.com/images/I/51b8ryqWlLL.jpg',
 ' https://images-na.ssl-images-amazon.com/images/I/513O5Q2Tk%2BL.jpg',
 ' https://images-na.ssl-images-amazon.com/images/I/51t2bxn-I2L.jpg',
 ' https://images-na.ssl-images-amazon.com/images/I/51vEXjR9xpL.jpg',
 ' https://images-na.ssl-images-amazon.com/images/I/61RCqdQ-MwL.jpg']

# Model

In [6]:
df_new.columns

Index(['category', 'description', 'title', 'brand', 'feature',
       'imageURLHighRes', 'main_ranking_3', 'numb_rankings',
       'clean_description', 'clean_feature', 'rank_binss'],
      dtype='object')

In [11]:
X = np.array(df_new['clean_description'] ,df_new['clean_feature'])

#  [df_new['title'],
#             df_new['clean_description'],
#             df_new['clean_feature'],
#             df_new['brand']]

y = np.array(df_new['rank_binss'].astype('int8'))

array(['  9H HARDNESS FOR MAXIMUM PROTECTION  9H harness tempered glass which is 3 times stronger than normal 3H hardness can effectively protect your Phone from unwanted scratches and friction from keys knife and other harmful factors. ANTI-SHATTER  By undergoing specialized tempered-glass treatment this screen protector is much more flexible and shatterproof. If the protector does break under extreme pressure it wont break into small sharp pieces keeping your phone and yourself out of danger. CLARITY AND HIGH-RESPONSE TOUCH PRESERVED  99.99% HD clarity color reproduction HD clear display natural watching experience maintains the original touch experience. WATERPROOF AND OIL-PROOF  High-density anti-fingerprint layer can prevent the water ; oil from scatter and the water ; oil can be wiped away very easily. 2.5D ROUND EDGE  The 2.5D round edge provides more comfortable handle feeling than the right angle edge and will never hurt your fingers.** NOTICE : screen protectors are manufactu

In [13]:
def tok_seo(X):
  ### Tokenizer
  tk = Tokenizer()
  tk.fit_on_texts(X)
  vocab_size = len(tk.word_index)

  print(f'There are {vocab_size} different words in your corpus')
  X_token = tk.texts_to_sequences(X)

  ### Pad the inputs
  X_pad = pad_sequences(X_token, dtype='float32', padding='post')

  return X_pad, vocab_size

X_pad, vocab_size = tok_seo(X)
print("X_pad.shape", X_pad.shape)

There are 17677 different words in your corpus
X_pad.shape (10422, 2940)


In [31]:
X_pad

array([[1.370e+02, 9.200e+01, 7.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [8.592e+03, 8.800e+01, 9.200e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [6.500e+01, 6.600e+01, 2.500e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [3.320e+02, 1.587e+03, 2.840e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.100e+01, 1.840e+02, 1.000e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00]], dtype=float32)

In [28]:
embedding_size = 100

model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1,
    input_length=2940, # Max_sentence_length (optional, for model summary)
    output_dim=embedding_size,
    mask_zero=True,))

model.add(layers.LSTM(30))
model.add(layers.Dense(11, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2940, 100)         1767800   
                                                                 
 lstm (LSTM)                 (None, 30)                15720     
                                                                 
 dense (Dense)               (None, 11)                341       
                                                                 
Total params: 1,783,861
Trainable params: 1,783,861
Non-trainable params: 0
_________________________________________________________________


In [14]:
y_cat = to_categorical(y)

In [15]:

X_train,X_test,y_cat_train,y_cat_test = train_test_split(X_pad,y_cat,train_size=0.3)

In [42]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_pad, y_cat, epochs=100, batch_size=512)
model.evaluate(X_pad,y_cat)

Epoch 1/100
21/21 [==============================] - 184s 9s/step - loss: 1.7493 - accuracy: 0.3848
Epoch 2/100
21/21 [==============================] - 218s 10s/step - loss: 1.6757 - accuracy: 0.4362
Epoch 3/100
21/21 [==============================] - 202s 10s/step - loss: 1.6239 - accuracy: 0.4729
Epoch 4/100
21/21 [==============================] - 181s 9s/step - loss: 1.5783 - accuracy: 0.4923
Epoch 5/100
21/21 [==============================] - 168s 8s/step - loss: 1.5377 - accuracy: 0.5041
Epoch 6/100
21/21 [==============================] - 165s 8s/step - loss: 1.4991 - accuracy: 0.5157
Epoch 7/100
21/21 [==============================] - 165s 8s/step - loss: 1.4642 - accuracy: 0.5326
Epoch 8/100
21/21 [==============================] - 166s 8s/step - loss: 1.4298 - accuracy: 0.5446
Epoch 9/100
21/21 [==============================] - 165s 8s/step - loss: 1.3922 - accuracy: 0.5576
Epoch 10/100
21/21 [==============================] - 164s 8s/step - loss: 1.3589 - accuracy: 0.58

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(X_pad,y_cat)

In [30]:
model_2 = Sequential()

model_2.add(layers.Embedding(input_dim=vocab_size+1,
                            input_length=2940,
                            output_dim=embedding_size,
                            mask_zero=True,))

model_2.add(layers.Bidirectional(layers.LSTM(120))

# model.add(layers.Dropout(0.3))
# model.add(layers.Dense(11, activation="softmax"))

model_2.add(layers.Dense(60,activation='relu'))

model_2.add(layers.Dense(11, activation="softmax"))


SyntaxError: ignored